In [1]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time
import random

client = OpenAI(
  api_key=""
)

# Đọc dữ liệu từ vnexpress_dataset.csv
df = pd.read_csv("./data/vnexpress_dataset.csv")

# Hàm gọi API để tạo tin giả
def generate_fake_news(row):
    try:
        content = row["Content"]
        prompt = f"""
        Đây là một bản tin thật từ báo chí chính thống:
        [TIN THẬT]: {content}

        Hãy tạo 3 bản tin giả dựa trên nội dung này. Các bản tin giả phải có nội dung hợp lý nhưng không đúng sự thật.
        Không sao chép nguyên văn từ tin thật mà hãy biến đổi câu chữ hoặc thêm thông tin gây hiểu lầm.

        Định dạng đầu ra:
        1. [Bản tin giả 1]
        2. [Bản tin giả 2]
        3. [Bản tin giả 3]

        Và không bao gồm các phần tử dưới đây:
        1. [Bản tin giả 1]
        2. [Bản tin giả 2]
        3. [Bản tin giả 3]

        Không đánh số đầu dòng
        """

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )

        fake_news = []
        for choice in response.choices:
            for fake_content in choice.message.content.split("\n"):
                if fake_content.strip():
                    fake_news.append({
                        "Title": row["Title"],
                        "Link": row["Link"],
                        "Views": row["Views"],
                        "Comments": row["Comments"],
                        "Content": fake_content.strip()
                    })

        # Filter out any potential duplicate of original content
        fake_news = [news for news in fake_news if news["Content"] != content]

        return fake_news
    except Exception as e:
        print(f"Error processing row: {e}")
        return []

# Số lượng worker để xử lý song song
MAX_WORKERS = 5

fake_news_list = []

# Xử lý song song với ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Tạo dictionary của các future với tham số đầu vào
    future_to_row = {executor.submit(generate_fake_news, row): i for i, row in df.iterrows()}

    # Xử lý kết quả khi hoàn thành
    for future in tqdm(concurrent.futures.as_completed(future_to_row), total=len(future_to_row), desc="Generating fake news"):
        try:
            results = future.result()
            fake_news_list.extend(results)

            # Giảm tải API bằng cách đợi một chút giữa các yêu cầu
            time.sleep(random.uniform(0.1, 0.5))
        except Exception as e:
            print(f"Exception occurred: {e}")

# Lưu vào file vnexpress_fake_dataset.csv
fake_df = pd.DataFrame(fake_news_list)
fake_df.to_csv("./data/vnexpress_fake_dataset_enhance.csv", index=False, encoding="utf-8-sig")

print(f"✅ Đã tạo và lưu {len(fake_news_list)} tin giả thành công vào vnexpress_fake_dataset.csv")

Generating fake news: 100%|██████████| 1672/1672 [51:48<00:00,  1.86s/it] 


✅ Đã tạo và lưu 8572 tin giả thành công vào vnexpress_fake_dataset.csv
